In [ ]:
This script will take the predictors and add a band called burn fraction from fire cci and save the new tif files. This will eventually be used to make a parquet file which is the training data frame

In [3]:
import os
import re
import numpy as np
import rasterio as rio
from rasterio.warp import reproject, Resampling
from pathlib import Path
from tqdm import tqdm
from calendar import monthrange
from datetime import date

# ---------------- PATHS ----------------
CEMS_DIR  = "/explore/nobackup/people/spotter5/clelland_fire_ml/training_e5l_cems_firecci"
FIRECCI_ANNUAL_DIR = "/explore/nobackup/people/spotter5/clelland_fire_ml/fire_cci_us"   # DOY rasters: YYYY.tif
OUT_DIR   = "/explore/nobackup/people/spotter5/clelland_fire_ml/training_e5l_cems_firecci_with_fraction"

os.makedirs(OUT_DIR, exist_ok=True)

# -------------- BANDS TO KEEP --------------
WANTED = [
    "DEM",
    "slope",
    "aspect",                     # will match 'aspect' or 'aspectrad' etc.
    "b1",                         # land cover (categorical, will be cast to float32 in output)
    "relative_humidity",
    "total_precipitation_sum",
    "temperature_2m",
    "temperature_2m_min",
    "temperature_2m_max",
    "build_up_index",
    "drought_code",
    "duff_moisture_code",
    "fine_fuel_moisture_code",
    "fire_weather_index",
    "initial_fire_spread_index",  # if your files use 'initial_spread_index', it’ll still match
]

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]", "", s.lower())

WANTED_NORM = [_norm(x) for x in WANTED]

# -------------- HELPERS --------------
name_re = re.compile(r"cems_e5l_firecci_(\d{4})_(\d{1,2})\.tif$", re.IGNORECASE)

def parse_year_month(path: Path):
    m = name_re.search(path.name)
    return (int(m.group(1)), int(m.group(2))) if m else None

def doy_range_for_month(year: int, month: int):
    """Return inclusive DOY start/end for given month in year, accounting for leap years."""
    first = date(year, month, 1)
    last_day = monthrange(year, month)[1]
    last = date(year, month, last_day)
    doy1 = (first - date(year, 1, 1)).days + 1
    doy2 = (last  - date(year, 1, 1)).days + 1
    return doy1, doy2

def map_band_indices_by_name(ds: rio.DatasetReader):
    mapping = {}
    descs = ds.descriptions  # tuple length = band count; may contain None
    for i, d in enumerate(descs, start=1):
        if d is None:
            d = f"B{i}"
        mapping[_norm(d)] = i
    return mapping, descs

# Cache annual FireCCI arrays per year to avoid re-reading 20+ times
_firecci_cache = {}

def get_firecci_year_arr(year: int):
    if year in _firecci_cache:
        return _firecci_cache[year]
    src_path = Path(FIRECCI_ANNUAL_DIR) / f"{year}.tif"
    if not src_path.exists():
        raise FileNotFoundError(f"Missing FireCCI annual DOY: {src_path}")
    with rio.open(src_path) as src:
        arr = src.read(1)         # DOY or 0/NoData
        info = {
            "crs": src.crs,
            "transform": src.transform,
            "nodata": src.nodata,
            "height": src.height,
            "width": src.width,
        }
    _firecci_cache[year] = (arr, info)
    return _firecci_cache[year]

def monthly_fraction_from_annual(year: int, month: int, tmpl_ds: rio.DatasetReader) -> np.ndarray:
    """
    From annual DOY raster, build uint8 mask for that month and average -> fraction on tmpl grid.
    1 = burned in month; 0 = not burned; 255 = nodata (ignored in averaging).
    """
    arr, info = get_firecci_year_arr(year)
    doy1, doy2 = doy_range_for_month(year, month)

    # Make a float view for NaN tests without copying unless needed
    a = arr.astype("float32", copy=False)
    src_nodata = info["nodata"]

    # Identify nodata
    is_nan = np.isnan(a)
    if src_nodata is not None and not np.isnan(src_nodata):
        is_nodata = is_nan | (a == src_nodata)
    else:
        is_nodata = is_nan

    # Burned this month: valid DOY within [doy1, doy2] and >0
    burned_this_month = (a >= doy1) & (a <= doy2) & (~is_nodata) & (a > 0)

    # Build mask
    mask = np.zeros(a.shape, dtype=np.uint8)  # default 0 (unburned)
    mask[burned_this_month] = 1               # burned in this month
    mask[is_nodata] = 255                     # nodata -> 255 so it’s ignored by averaging

    # Reproject/aggregate to template grid
    frac = np.full((tmpl_ds.height, tmpl_ds.width), np.nan, dtype=np.float32)
    reproject(
        source=mask,
        destination=frac,
        src_transform=info["transform"],
        src_crs=info["crs"],
        src_nodata=255,                 # tell reproject to ignore 255
        dst_transform=tmpl_ds.transform,
        dst_crs=tmpl_ds.crs,
        dst_nodata=np.nan,              # keep NaNs in memory; we won’t set nodata on write
        resampling=Resampling.average,  # 0/1 average -> monthly burned fraction
        num_threads=0,                  # ALL_CPUS
    )
    return frac  # 0..1, NaN where no contributing data


# -------------- MAIN --------------
tifs = sorted(Path(CEMS_DIR).glob("cems_e5l_firecci_*.tif"))
if not tifs:
    print(f"No CEMS files found in {CEMS_DIR}")

for src_path in tqdm(tifs):
    ym = parse_year_month(src_path)
    if ym is None:
        print(f"[SKIP name] {src_path.name}")
        continue
    year, month = ym

    out_path = Path(OUT_DIR) / src_path.name.replace(".tif", "_with_fraction.tif")
    if out_path.exists():
        print(f"[SKIP exists] {out_path.name}")
        continue

    with rio.open(src_path) as ds:
        band_map, _ = map_band_indices_by_name(ds)

        # Select bands in the requested order (best-effort name matching)
        keep_indices, keep_names = [], []
        for want_norm, want_orig in zip(WANTED_NORM, WANTED):
            if want_norm in band_map:
                keep_indices.append(band_map[want_norm])
                keep_names.append(want_orig)
                continue
            # partial (handles 'aspect' vs 'aspectrad', etc.)
            match_idx = None
            for k_norm, idx in band_map.items():
                if want_norm in k_norm or k_norm in want_norm:
                    match_idx = idx
                    break
            if match_idx is not None:
                keep_indices.append(match_idx)
                keep_names.append(want_orig)
            else:
                print(f"[WARN] {src_path.name}: could not find band like '{want_orig}'")

        if not keep_indices:
            print(f"[SKIP no-bands] {src_path.name}")
            continue

        # Read selected bands (cast to float32 for a single dtype stack)
        data_list = [ds.read(bi).astype(np.float32) for bi in keep_indices]

        # Build monthly fraction from annual DOY and reproject/aggregate to this grid
        frac = monthly_fraction_from_annual(year, month, ds).astype(np.float32)
        data_list.append(frac)

        out_arr = np.stack(data_list, axis=0)  # (bands, H, W)

        # Profile for output
        profile = ds.profile.copy()
        profile.update(
            dtype="float32",
            count=out_arr.shape[0],
            compress="LZW",
            tiled=True,
            blockxsize=256,
            blockysize=256,
            BIGTIFF="IF_SAFER",
            nodata=None,  # leave NaNs; many stacks prefer this for float data
        )

        # Write output & band descriptions
        with rio.open(out_path, "w", **profile) as dst:
            dst.write(out_arr)
            descs = keep_names + ["fraction"]
            for i, nm in enumerate(descs, start=1):
                dst.set_band_description(i, nm)

    print(f"[OK] {out_path.name}  (kept {len(keep_names)} bands + fraction)")


  0%|          | 1/240 [00:47<3:09:39, 47.61s/it]

[OK] cems_e5l_firecci_2001_1_with_fraction.tif  (kept 15 bands + fraction)


  1%|          | 2/240 [01:32<3:03:31, 46.27s/it]

[OK] cems_e5l_firecci_2001_10_with_fraction.tif  (kept 15 bands + fraction)


  1%|▏         | 3/240 [02:17<2:59:14, 45.38s/it]

...

[OK] cems_e5l_firecci_2019_8_with_fraction.tif  (kept 15 bands + fraction)


 95%|█████████▌| 228/240 [3:05:53<10:42, 53.58s/it]

[OK] cems_e5l_firecci_2019_9_with_fraction.tif  (kept 15 bands + fraction)


 95%|█████████▌| 228/240 [3:05:55<09:47, 48.93s/it]


FileNotFoundError: Missing FireCCI annual DOY: /explore/nobackup/people/spotter5/clelland_fire_ml/fire_cci_us/2020.tif

In [ ]:
't'

Make parquet file

In [2]:
import os
import re
import numpy as np
import pandas as pd
import rasterio as rio
from rasterio.windows import Window
from rasterio.warp import transform as rio_transform
from pathlib import Path
from tqdm import tqdm

import pyarrow as pa
import pyarrow.parquet as pq

# ================== CONFIG ==================
IN_DIR   = "/explore/nobackup/people/spotter5/clelland_fire_ml/training_e5l_cems_firecci_with_fraction"
OUT_DATASET_DIR = "/explore/nobackup/people/spotter5/clelland_fire_ml/parquet_cems_with_fraction_dataset"  # <- single dataset root
os.makedirs(OUT_DATASET_DIR, exist_ok=True)

# If your TIFF CRS is projected (e.g., EPSG:3413) but you want true lat/lon degrees:
REPROJECT_TO_EPSG4326 = True

# Band names are taken from descriptions in the TIFFs
# (We’ll sanitize to safe column names and ensure uniqueness.)
def sanitize_names(names):
    seen = {}
    out = []
    for n in names:
        if n is None or str(n).strip() == "":
            n = "band"
        n0 = re.sub(r"[^a-zA-Z0-9_]", "_", str(n).strip())
        n0 = re.sub(r"_+", "_", n0).strip("_")
        if n0 == "":
            n0 = "band"
        if n0 in seen:
            seen[n0] += 1
            n0 = f"{n0}_{seen[n0]}"
        else:
            seen[n0] = 1
        out.append(n0)
    return out

name_re = re.compile(r"cems_e5l_firecci_(\d{4})_(\d{1,2})", re.IGNORECASE)
def parse_year_month(fname: str):
    m = name_re.search(fname)
    if not m: return None, None
    return int(m.group(1)), int(m.group(2))

def windows_for_dataset(ds: rio.DatasetReader):
    """Iterate using native block windows when possible (COG-friendly)."""
    try:
        for _, w in ds.block_windows(1):
            yield w
    except Exception:
        tile = 1024
        for row_off in range(0, ds.height, tile):
            height = min(tile, ds.height - row_off)
            for col_off in range(0, ds.width, tile):
                width = min(tile, ds.width - col_off)
                yield Window(col_off, row_off, width, height)

def append_chunk_to_dataset(df: pd.DataFrame, root: str):
    """Append a chunk to a partitioned Parquet dataset: year=/month= subdirs."""
    # PyArrow: convert then write into dataset with partition columns
    table = pa.Table.from_pandas(df, preserve_index=False)
    pq.write_to_dataset(
        table,
        root_path=root,
        partition_cols=["year", "month"],  # directories year=YYYY/month=MM
        # engine will create/append files inside the relevant partition folders
        use_dictionary=False
    )

# ================== MAIN ==================
tifs = sorted(Path(IN_DIR).glob("cems_e5l_firecci_*with_fraction.tif"))
if not tifs:
    tifs = sorted(Path(IN_DIR).glob("cems_e5l_firecci_*.tif"))

if not tifs:
    raise FileNotFoundError(f"No CEMS TIFFs found in {IN_DIR}")

# Optional: track a canonical column order from the first file for consistency
canonical_cols = None

for tif in tqdm(tifs, desc="Building partitioned Parquet dataset"):
    year, month = parse_year_month(tif.name)
    if year is None:
        print(f"[SKIP] {tif.name} (no YYYY_M in name)")
        continue

    with rio.open(tif) as ds:
        # Band names
        band_names = list(ds.descriptions)
        if not any(band_names):
            band_names = [f"B{i}" for i in range(1, ds.count + 1)]
        safe_names = sanitize_names(band_names)

        # Establish canonical column order (bands first) once
        if canonical_cols is None:
            canonical_cols = safe_names + ["longitude", "latitude", "year", "month"]

        # Process in windows and append per-window
        for w in windows_for_dataset(ds):
            data = ds.read(indexes=list(range(1, ds.count + 1)), window=w).astype(np.float32)  # (bands, h, w)
            h, w_cols = data.shape[1], data.shape[2]

            # Row/col grid for this window
            rows = np.arange(int(w.row_off), int(w.row_off + h))
            cols = np.arange(int(w.col_off), int(w.col_off + w_cols))
            rr, cc = np.meshgrid(rows, cols, indexing="ij")

            # Pixel center coords in native CRS
            xs, ys = rio.transform.xy(ds.transform, rr, cc, offset="center")
            xs = np.asarray(xs, dtype=np.float64).reshape(-1)
            ys = np.asarray(ys, dtype=np.float64).reshape(-1)

            # Reproject to lon/lat if desired
            if REPROJECT_TO_EPSG4326 and ds.crs is not None and ds.crs.to_string().upper() not in ("EPSG:4326", "OGC:CRS84"):
                # transform expects 1-D arrays; returns lists
                lons, lats = rio_transform(ds.crs, "EPSG:4326", xs, ys)
                lons = np.array(lons, dtype=np.float64)
                lats = np.array(lats, dtype=np.float64)
            else:
                # Already geographic or CRS unknown; treat xs=longitude, ys=latitude as-is
                lons, lats = xs, ys

            # Flatten bands to rows
            arr2d = data.reshape(ds.count, -1).T  # (h*w, bands)
            df = pd.DataFrame(arr2d, columns=safe_names)

            # Add coords + partitions
            df["longitude"] = lons
            df["latitude"]  = lats
            df["year"]      = year
            df["month"]     = month

            # Align column order to canonical set (in case bands vary)
            for col in canonical_cols:
                if col not in df.columns:
                    df[col] = np.nan  # ensure schema stability
            df = df[canonical_cols]

            # Append window-chunk to dataset
            append_chunk_to_dataset(df, OUT_DATASET_DIR)

print(f"✅ Done. Parquet dataset at:\n{OUT_DATASET_DIR}\n(partitioned by year=/month=)")


Building partitioned Parquet dataset: 100%|██████████| 228/228 [27:07<00:00,  7.14s/it]

✅ Done. Parquet dataset at:
/explore/nobackup/people/spotter5/clelland_fire_ml/parquet_cems_with_fraction_dataset
(partitioned by year=/month=)


In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# =====================================================
# LOAD DATA
# =====================================================
PARQUET_DATASET = "/explore/nobackup/people/spotter5/clelland_fire_ml/parquet_cems_with_fraction_dataset"
df = pd.read_parquet(PARQUET_DATASET)

# Drop missing targets
df = df.dropna(subset=["fraction"]).copy()

# =====================================================
# BINNING FOR BALANCED SAMPLING
# =====================================================
bins = [0.0, 0.0, 0.33, 0.66, 1.0]
labels = [0, 1, 2, 3]
df["bin"] = pd.cut(df["fraction"], bins=bins, labels=labels, include_lowest=True)
df["bin"] = df["bin"].astype(float)
df = df.dropna(subset=["bin"]).copy()

print("---- Counts per fraction bin (before sampling) ----")
print(df["bin"].value_counts().sort_index())

# Balance by sampling equally across bins
min_count = df["bin"].value_counts().min()
balanced = (
    df.groupby("bin", group_keys=False)
      .apply(lambda x: x.sample(min_count, random_state=42))
      .reset_index(drop=True)
)

print("\n---- Counts per fraction bin (after balancing) ----")
print(balanced["bin"].value_counts().sort_index())

# =====================================================
# FEATURES & TARGET
# =====================================================
predictors = [c for c in df.columns if c not in ["fraction", "year", "month", "latitude", "longitude", "bin"]]
cat_col = "b1" if "b1" in predictors else None

X = balanced[predictors].copy()
y = balanced["fraction"].astype(np.float32)

# Keep b1 as categorical if integer-coded
if cat_col is not None:
    if np.issubdtype(X[cat_col].dtype, np.number):
        print(f"\nTreating '{cat_col}' as categorical (integer codes).")
        cat_features = [cat_col]
    else:
        print(f"\nConverting '{cat_col}' to categorical codes.")
        X[cat_col] = X[cat_col].astype("category").cat.codes
        cat_features = [cat_col]
else:
    cat_features = []

# =====================================================
# SPLIT TRAIN/VALIDATION
# =====================================================
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nTraining samples: {len(X_train):,}")
print(f"Validation samples: {len(X_val):,}")

# =====================================================
# LIGHTGBM REGRESSOR
# =====================================================
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
val_data   = lgb.Dataset(X_val, label=y_val, reference=train_data, categorical_feature=cat_features)

params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 64,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "min_data_in_leaf": 50,
    "n_jobs": -1,
    "verbose": -1,
}

evals_result = {}
model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, val_data],
    valid_names=["train", "validation"],
    num_boost_round=1000,
    early_stopping_rounds=50,
    evals_result=evals_result,
    verbose_eval=50
)

# =====================================================
# PLOT TRAIN vs VALIDATION RMSE
# =====================================================
plt.figure(figsize=(8,5))
plt.plot(evals_result["train"]["rmse"], label="Train RMSE")
plt.plot(evals_result["validation"]["rmse"], label="Validation RMSE")
plt.xlabel("Boosting Rounds")
plt.ylabel("RMSE")
plt.title("LightGBM Training vs Validation RMSE")
plt.legend()
plt.grid(True)
plt.show()

# =====================================================
# FINAL METRICS
# =====================================================
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)
r2 = 1 - np.sum((y_val - y_pred)**2) / np.sum((y_val - np.mean(y_val))**2)
print(f"\nValidation RMSE: {rmse:.4f}")
print(f"Validation R²: {r2:.4f}")


/home/spotter5/.conda/envs/xgboost_gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
